# Real-world Data Wrangling  
**Fullname:** Nguyen Van Khang  
**ID:** 23IT121  
**Class:** 23SE2  

## 1. Gather data

### **1.1.** Problem Statement

## This project aims to practice my skills of data wrangling using real-world datasets.

### In this dataset, I would like to answer the following research questions:

#### Question 1: Điểm thi cao nhất của môn Hoá năm thi đó

#### Question 2: Thí sinh nào có tổng điểm cao nhất?

#### Question 3: Trung bình điểm môn Ngữ Văn của tất cả thí sinh là bao nhiêu?

In [22]:

import pandas as pd
import numpy as np
# Dung thu vien Pandas de doc file CSV
df = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2020.csv')
df1 = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2022.csv')
df2 = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2024.csv')

In [23]:
# Q1:Diem Hóa cao nhất trong dữ liệu
def get_max_score(df, year):
    column_name = "hoa_hoc"
    if column_name in df.columns:
        max_score = df[column_name].dropna().max()
        print(f"Điểm cao nhất môn Hóa năm {year}: {max_score}")
    else:
        print(f"Không tìm thấy cột '{column_name}' trong dữ liệu năm {year}.")
# Gọi hàm cho từng năm
get_max_score(df, 2020)
get_max_score(df1, 2022)
get_max_score(df2, 2024)


Điểm cao nhất môn Hóa năm 2020: 10.0
Điểm cao nhất môn Hóa năm 2022: 10.0
Điểm cao nhất môn Hóa năm 2024: 10.0


In [24]:
import pandas as pd

def chuan_hoa_ten_cot(df):
    """Chuẩn hóa tên cột về chữ thường, không dấu, thay khoảng trắng bằng '_'."""
    mapping = {
        "SBD": "sbd", "Toán": "toan", "Ngữ Văn": "ngu_van", "Vật Lý": "vat_li",
        "Hóa Học": "hoa_hoc", "Sinh Học": "sinh_hoc", "Lịch Sử": "lich_su",
        "Địa Lý": "dia_li", "GDCD": "gdcd", "Ngoại Ngữ": "ngoai_ngu"
    }
    df = df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"))
    df = df.rename(columns=mapping)  # Áp dụng ánh xạ cột
    return df

def tim_thi_sinh_top(df, nam):
    df = chuan_hoa_ten_cot(df)  # Chuẩn hóa tên cột

    if "nam" in df.columns:
        df = df[df["nam"] == nam]  # Chỉ lấy dữ liệu của năm cần tìm

    if df.empty:
        print(f"⚠️ Không có dữ liệu cho năm {nam}!")
        return

    # Danh sách các môn thi hợp lệ
    mon_thi = ["toan", "ngu_van", "vat_li", "hoa_hoc", "sinh_hoc",
               "lich_su", "dia_li", "gdcd", "ngoai_ngu"]
    mon_thi = [mon for mon in mon_thi if mon in df.columns]

    # Chuyển đổi điểm về dạng số để đảm bảo tính toán đúng
    df[mon_thi] = df[mon_thi].apply(pd.to_numeric, errors="coerce")

    df = df.copy()
    df["tong_diem"] = df[mon_thi].sum(axis=1, skipna=True)

    # Kiểm tra lại tổng điểm có giá trị hợp lý không
    print(f"📊 Tổng điểm cao nhất của năm {nam}: {df['tong_diem'].max()}")

    # Tìm thí sinh có tổng điểm cao nhất
    max_score = df["tong_diem"].max()
    thisinh_max = df[df["tong_diem"] == max_score]

    # In kết quả
    print(f"🏆 Thí sinh có tổng điểm cao nhất năm {nam}:")
    print(thisinh_max[["sbd"] + mon_thi + ["tong_diem"]])
    print("=" * 50)

# Chạy lại cho 2020
tim_thi_sinh_top(df, 2020)
tim_thi_sinh_top(df1, 2022)
tim_thi_sinh_top(df2, 2024)

📊 Tổng điểm cao nhất của năm 2020: 56.7
🏆 Thí sinh có tổng điểm cao nhất năm 2020:
             sbd  toan  ngu_van  vat_li  hoa_hoc  sinh_hoc  lich_su  dia_li  \
235741  25014073   9.4      9.0     NaN      NaN       NaN     10.0    9.75   

        gdcd  ngoai_ngu  tong_diem  
235741  9.75        8.8       56.7  
📊 Tổng điểm cao nhất của năm 2022: 56.85
🏆 Thí sinh có tổng điểm cao nhất năm 2022:
             sbd  toan  ngu_van  vat_li  hoa_hoc  sinh_hoc  lich_su  dia_li  \
474053  27005570   9.2     9.25     NaN      NaN       NaN     10.0    9.25   

        gdcd  ngoai_ngu  tong_diem  
474053  9.75        9.4      56.85  
📊 Tổng điểm cao nhất của năm 2024: 57.849999999999994
🏆 Thí sinh có tổng điểm cao nhất năm 2024:
             sbd  toan  ngu_van  vat_li  hoa_hoc  sinh_hoc  lich_su  dia_li  \
33906    1034208   8.8     9.25     NaN      NaN       NaN     10.0    10.0   
515167  27004000   8.8     9.75     NaN      NaN       NaN     10.0    10.0   

        gdcd  ngoai_ngu  tong_di

In [25]:
# Trung bình điểm môn Ngữ Văn của tất cả thí sinh là bao nhiêu?
def tinh_trung_binh_van(df, nam):
    # Đảm bảo cột "Ngữ Văn" tồn tại trước khi tính toán
    if "Ngữ Văn" in df.columns:
        avg_van = df["Ngữ Văn"].mean()
        print(f"Trung bình điểm môn Ngữ Văn năm {nam}: {avg_van:.2f}")
    else:
        print(f" Không tìm thấy cột 'Ngữ Văn' trong dữ liệu năm {nam}.")

# Gọi hàm cho từng năm
tinh_trung_binh_van(df, 2020)
tinh_trung_binh_van(df1, 2022)
tinh_trung_binh_van(df2, 2024)


 Không tìm thấy cột 'Ngữ Văn' trong dữ liệu năm 2020.
 Không tìm thấy cột 'Ngữ Văn' trong dữ liệu năm 2022.
 Không tìm thấy cột 'Ngữ Văn' trong dữ liệu năm 2024.


In [26]:
# Hien thi 5 hang dau cua Dataframe thu 2
df1.head()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
0,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.0,8.75
1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.5,8.25
2,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.5,8.75
3,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.5,7.50
4,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.5,8.25


### **1.2.** Gather at least two datasets using two different data gathering methods


In [27]:
# Dung thu vien Pandas de doc file CSV
df = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2020.csv')
df1 = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2022.csv')
df2 = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2024.csv')

In [28]:
# Hien thi 5 hang dau cua Dataframe thu 1
df.head()

,Unnamed: 0,sbd,ten,ngay_sinh,gioi_tinh,toan,ngu_van,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ngoai_ngu
0,0,18014547,NaN,NaN,NaN,6.4,6.75,NaN,NaN,NaN,4.75,7.00,6.50,4.2
1,1,18014530,NaN,NaN,NaN,7.6,6.00,NaN,NaN,NaN,3.75,7.75,7.75,2.8
2,2,18014521,NaN,NaN,NaN,4.8,4.75,NaN,NaN,NaN,4.00,6.50,NaN,NaN
3,3,18014517,NaN,NaN,NaN,8.0,7.00,NaN,NaN,NaN,8.25,8.00,9.50,5.8
4,4,18014523,NaN,NaN,NaN,8.2,6.50,8.0,8.5,5.0,NaN,NaN,NaN,4.0


In [29]:
# Hien thi 5 hang dau cua Dataframe thu 2
df1.head()


,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
0,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.0,8.75
1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.5,8.25
2,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.5,8.75
3,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.5,7.50
4,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.5,8.25


In [30]:
# Hien thi 5 hang dau cua Dataframe thu 3
df2.head()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ma_ngoai_ngu
0,1000001,8.4,6.75,8.0,6.0,5.25,5.0,NaN,NaN,NaN,N1
1,1000002,8.6,8.50,7.2,NaN,NaN,NaN,7.25,6.00,8.00,N1
2,1000003,8.2,8.75,8.2,NaN,NaN,NaN,7.25,7.25,8.75,N1
3,1000004,4.8,8.50,7.4,NaN,NaN,NaN,7.00,6.00,7.50,N1
4,1000005,8.6,9.00,7.8,NaN,NaN,NaN,9.00,8.75,8.50,N1


## 2. Assess data

In [31]:
# Hien thi thong tin cua Dataframe thu 1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870486 entries, 0 to 870485
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  870486 non-null  int64  
 1   sbd         870486 non-null  int64  
 2   ten         0 non-null       float64
 3   ngay_sinh   0 non-null       float64
 4   gioi_tinh   0 non-null       float64
 5   toan        866581 non-null  float64
 6   ngu_van     856565 non-null  float64
 7   vat_li      293287 non-null  float64
 8   hoa_hoc     295536 non-null  float64
 9   sinh_hoc    290377 non-null  float64
 10  lich_su     568581 non-null  float64
 11  dia_li      555072 non-null  float64
 12  gdcd        482980 non-null  float64
 13  ngoai_ngu   772098 non-null  float64
dtypes: float64(12), int64(2)
memory usage: 93.0 MB


In [32]:
# Hien thi thong tin cua Dataframe thu 2
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995441 entries, 0 to 995440
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   sbd        995441 non-null  int64  
 1   toan       982726 non-null  float64
 2   ngu_van    981407 non-null  float64
 3   ngoai_ngu  870609 non-null  float64
 4   vat_li     325523 non-null  float64
 5   hoa_hoc    327367 non-null  float64
 6   sinh_hoc   322198 non-null  float64
 7   lich_su    659662 non-null  float64
 8   dia_li     657421 non-null  float64
 9   gdcd       554343 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 75.9 MB


In [33]:
# Hien thi thong tin cua Dataframe thu 3
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061605 entries, 0 to 1061604
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   sbd           1061605 non-null  int64  
 1   toan          1045613 non-null  float64
 2   ngu_van       1050101 non-null  float64
 3   ngoai_ngu     912705 non-null   float64
 4   vat_li        345615 non-null   float64
 5   hoa_hoc       346518 non-null   float64
 6   sinh_hoc      342378 non-null   float64
 7   lich_su       706214 non-null   float64
 8   dia_li        704682 non-null   float64
 9   gdcd          583609 non-null   float64
 10  ma_ngoai_ngu  912705 non-null   object 
dtypes: float64(9), int64(1), object(1)
memory usage: 89.1+ MB


In [34]:
# Dua ra tong quan ve du lieu cua Dataframe thu 1
df.describe()

,Unnamed: 0,sbd,ten,ngay_sinh,gioi_tinh,toan,ngu_van,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ngoai_ngu
count,870486.00000,8.704860e+05,0.0,0.0,0.0,866581.000000,856565.000000,293287.000000,295536.000000,290377.000000,568581.000000,555072.000000,482980.000000,772098.000000
mean,435242.50000,2.800845e+07,NaN,NaN,NaN,6.662271,6.601825,6.718001,6.705225,5.588041,5.181097,6.773215,8.125164,4.568337
std,251287.80755,1.898843e+07,NaN,NaN,NaN,1.819042,1.248463,1.497900,1.607924,1.350572,1.598987,1.171702,1.090310,1.808446
min,0.00000,1.000001e+06,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,217621.25000,1.200657e+07,NaN,NaN,NaN,5.400000,6.000000,5.750000,5.500000,4.750000,4.000000,6.000000,7.500000,3.200000
50%,435242.50000,2.801803e+07,NaN,NaN,NaN,7.000000,6.750000,7.000000,7.000000,5.500000,5.000000,7.000000,8.250000,4.200000
75%,652863.75000,4.400927e+07,NaN,NaN,NaN,8.000000,7.500000,7.750000,8.000000,6.500000,6.250000,7.500000,9.000000,5.600000
max,870485.00000,6.400582e+07,NaN,NaN,NaN,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [35]:
# Duas ra tong quan ve du lieu cua Dataframe thu 2
df1.describe()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
count,9.954410e+05,982726.000000,981407.000000,870609.000000,325523.000000,327367.000000,322198.000000,659662.000000,657421.000000,554343.000000
mean,2.759692e+07,6.466254,6.507746,5.157686,6.723779,6.703898,5.019109,6.343930,6.677929,8.033266
std,1.917092e+07,1.675435,1.379602,1.949547,1.441925,1.600393,1.448270,1.612879,1.215013,1.067272
min,1.000001e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000217e+07,5.400000,5.580000,3.600000,5.750000,5.500000,4.000000,5.250000,5.750000,7.500000
50%,2.801886e+07,6.800000,6.500000,4.800000,7.000000,7.000000,4.750000,6.500000,6.750000,8.250000
75%,4.400582e+07,7.800000,7.500000,6.600000,7.750000,8.000000,6.000000,7.500000,7.500000,8.750000
max,6.400659e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [36]:
# Duas ra tong quan ve du lieu cua Dataframe thu 3
df2.describe()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
count,1.061605e+06,1.045613e+06,1.050101e+06,912705.000000,345615.000000,346518.00000,342378.000000,706214.000000,704682.000000,583609.000000
mean,2.734354e+07,6.447309e+00,7.231146e+00,5.521938,6.666866,6.68083,6.284079,6.570343,7.193541,8.156503
std,1.919442e+07,1.556900e+00,1.330057e+00,1.880657,1.631129,1.64147,1.244849,1.462804,1.283945,1.042644
min,1.000001e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,9.006324e+06,5.400000e+00,6.500000e+00,4.000000,5.500000,5.50000,5.500000,5.500000,6.500000,7.500000
50%,2.800794e+07,6.800000e+00,7.500000e+00,5.200000,7.000000,6.75000,6.250000,6.500000,7.250000,8.250000
75%,4.400492e+07,7.600000e+00,8.250000e+00,7.000000,8.000000,8.00000,7.250000,7.500000,8.000000,9.000000
max,6.400729e+07,9.800000e+00,1.000000e+01,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000


# **3. Clean data**

 **Problem 1: Check for Missing or Redundant Data (Missing Values)**
 
 **Problem 2: Remove Unnecessary Columns and Rename Columns**
 
 **Problem 3: Standardize Data**
 
 **Problem 4: Handle Anomalous Data**

### Problem 1: Check for Missing or Redundant Data (Missing Values)

In [37]:
import pandas as pd
import numpy as np
# Dung thu vien Pandas de doc file CSV
df = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2020.csv')
# Xoa cot 'ten', 'ngay_sinh', 'gioi_tinh' 
columns_to_drop = ["ten", "ngay_sinh","gioi_tinh"]
# Xoa cot neu cot do ton tai trong dataframe
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
# Chuyen gia tri NaN thanh 0
df = df.fillna(0)
df


,Unnamed: 0,sbd,toan,ngu_van,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ngoai_ngu
0,0,18014547,6.4,6.75,0.0,0.0,0.0,4.75,7.00,6.50,4.2
1,1,18014530,7.6,6.00,0.0,0.0,0.0,3.75,7.75,7.75,2.8
2,2,18014521,4.8,4.75,0.0,0.0,0.0,4.00,6.50,0.00,0.0
3,3,18014517,8.0,7.00,0.0,0.0,0.0,8.25,8.00,9.50,5.8
4,4,18014523,8.2,6.50,8.0,8.5,5.0,0.00,0.00,0.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...
870481,870481,17014593,4.6,4.00,0.0,0.0,0.0,3.75,6.75,6.25,2.0
870482,870482,17014597,7.4,8.00,0.0,0.0,0.0,5.50,7.50,9.50,2.6
870483,870483,17014595,8.2,8.75,0.0,0.0,0.0,4.50,6.50,8.50,6.2
870484,870484,17014598,4.6,6.50,0.0,0.0,0.0,4.50,7.50,9.00,3.6


In [38]:
import pandas as pd
import numpy as np
# Dung thu vien Pandas de doc file CSV
df1 = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2022.csv')
# Xoa cot 'ten', 'ngay_sinh', 'gioi_tinh'
columns_to_drop = ["ten", "ngay_sinh","gioi_tinh"]
# Xoa cot neu cot do ton tai trong dataframe
df1 = df1.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
# Chuyen gia tri NaN thanh 0    
df1 = df1.fillna(0)
df1

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
0,1000001,3.6,5.00,4.0,0.0,0.0,0.0,2.75,6.00,8.75
1,1000002,8.4,6.75,7.6,0.0,0.0,0.0,8.50,7.50,8.25
2,1000003,5.8,7.50,5.0,0.0,0.0,0.0,7.25,5.50,8.75
3,1000004,7.4,7.50,8.6,0.0,0.0,0.0,7.50,6.50,7.50
4,1000005,7.2,8.50,9.0,0.0,0.0,0.0,8.00,8.50,8.25
...,...,...,...,...,...,...,...,...,...,...
995436,64006584,8.4,6.75,4.6,0.0,0.0,0.0,6.50,6.75,9.00
995437,64006585,5.6,6.50,2.8,0.0,0.0,0.0,6.25,6.75,8.50
995438,64006586,5.8,6.00,6.6,0.0,0.0,0.0,7.25,8.00,8.00
995439,64006587,7.6,6.75,7.0,0.0,0.0,0.0,8.75,7.25,9.75


In [39]:
import pandas as pd
import numpy as np
# Dung thu vien Pandas de doc file CSV
df2 = pd.read_csv(r'D:\dataset\messy\diem_thi_thpt_2024.csv')
# Chuyen gia tri NaN thanh 0
df2 = df2.fillna(0)
# Doi ten cot
df2


,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ma_ngoai_ngu
0,1000001,8.4,6.75,8.0,6.00,5.25,5.0,0.00,0.00,0.00,N1
1,1000002,8.6,8.50,7.2,0.00,0.00,0.0,7.25,6.00,8.00,N1
2,1000003,8.2,8.75,8.2,0.00,0.00,0.0,7.25,7.25,8.75,N1
3,1000004,4.8,8.50,7.4,0.00,0.00,0.0,7.00,6.00,7.50,N1
4,1000005,8.6,9.00,7.8,0.00,0.00,0.0,9.00,8.75,8.50,N1
...,...,...,...,...,...,...,...,...,...,...,...
1061600,64007282,4.2,6.00,3.0,0.00,0.00,0.0,5.25,6.25,7.50,N1
1061601,64007283,5.2,6.00,3.6,0.00,0.00,0.0,6.25,5.75,7.75,N1
1061602,64007284,7.2,8.00,7.0,6.25,7.00,8.0,0.00,0.00,0.00,N1
1061603,64007285,5.2,5.25,4.4,0.00,0.00,0.0,4.75,6.50,8.75,N1


### **Problem 2: Remove Unnecessary Columns and Rename Column Name**


In [40]:
# Doi ten cot cuar Dataframe thu 2
df1=df1.rename(columns={
    "Unnamed: 0": "STT",
    "sbd": "SBD",
    "toan": "Toán",
    "ngu_van": "Ngữ Văn",
    "vat_li": "Vật Lý",
    "hoa_hoc": "Hóa Học",
    "sinh_hoc": "Sinh Học",
    "lich_su": "Lịch Sử",
    "dia_li": "Địa Lý",
    "gdcd": "GDCD",
    "ngoai_ngu": "Ngoại Ngữ"
})
df1.columns

Index(['SBD', 'Toán', 'Ngữ Văn', 'Ngoại Ngữ', 'Vật Lý', 'Hóa Học', 'Sinh Học',
       'Lịch Sử', 'Địa Lý', 'GDCD'],
      dtype='object')

In [41]:
# DOI ten cot cua Dataframe thu 3
df2=df2.rename(columns={
    "sbd": "SBD",
    "toan": "Toán",
    "ngu_van": "Ngữ Văn",
    "vat_li": "Vật Lý",
    "hoa_hoc": "Hóa Học",
    "sinh_hoc": "Sinh Học",
    "lich_su": "Lịch Sử",
    "dia_li": "Địa Lý",
    "gdcd": "GDCD",
    "ngoai_ngu": "Ngoại Ngữ",
    "ma_ngoai_ngu": "Mã Ngoại Ngữ"
})
df2

,SBD,Toán,Ngữ Văn,Ngoại Ngữ,Vật Lý,Hóa Học,Sinh Học,Lịch Sử,Địa Lý,GDCD,Mã Ngoại Ngữ
0,1000001,8.4,6.75,8.0,6.00,5.25,5.0,0.00,0.00,0.00,N1
1,1000002,8.6,8.50,7.2,0.00,0.00,0.0,7.25,6.00,8.00,N1
2,1000003,8.2,8.75,8.2,0.00,0.00,0.0,7.25,7.25,8.75,N1
3,1000004,4.8,8.50,7.4,0.00,0.00,0.0,7.00,6.00,7.50,N1
4,1000005,8.6,9.00,7.8,0.00,0.00,0.0,9.00,8.75,8.50,N1
...,...,...,...,...,...,...,...,...,...,...,...
1061600,64007282,4.2,6.00,3.0,0.00,0.00,0.0,5.25,6.25,7.50,N1
1061601,64007283,5.2,6.00,3.6,0.00,0.00,0.0,6.25,5.75,7.75,N1
1061602,64007284,7.2,8.00,7.0,6.25,7.00,8.0,0.00,0.00,0.00,N1
1061603,64007285,5.2,5.25,4.4,0.00,0.00,0.0,4.75,6.50,8.75,N1


In [42]:
# DOi ten cot cua Dataframe thu 1
df=df.rename(columns={
    "Unnamed: 0": "STT",
    "sbd": "SBD",
    "toan": "Toán",
    "ngu_van": "Ngữ Văn",
    "vat_li": "Vật Lý",
    "hoa_hoc": "Hóa Học",
    "sinh_hoc": "Sinh Học",
    "lich_su": "Lịch Sử",
    "dia_li": "Địa Lý",
    "gdcd": "GDCD",
    "ngoai_ngu": "Ngoại Ngữ"
})
df

,STT,SBD,Toán,Ngữ Văn,Vật Lý,Hóa Học,Sinh Học,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ
0,0,18014547,6.4,6.75,0.0,0.0,0.0,4.75,7.00,6.50,4.2
1,1,18014530,7.6,6.00,0.0,0.0,0.0,3.75,7.75,7.75,2.8
2,2,18014521,4.8,4.75,0.0,0.0,0.0,4.00,6.50,0.00,0.0
3,3,18014517,8.0,7.00,0.0,0.0,0.0,8.25,8.00,9.50,5.8
4,4,18014523,8.2,6.50,8.0,8.5,5.0,0.00,0.00,0.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...
870481,870481,17014593,4.6,4.00,0.0,0.0,0.0,3.75,6.75,6.25,2.0
870482,870482,17014597,7.4,8.00,0.0,0.0,0.0,5.50,7.50,9.50,2.6
870483,870483,17014595,8.2,8.75,0.0,0.0,0.0,4.50,6.50,8.50,6.2
870484,870484,17014598,4.6,6.50,0.0,0.0,0.0,4.50,7.50,9.00,3.6


 ### **Problem 3: Standalize Data**


Dữ liệu đã phù hợp nên không cần chuẩn hoá

 ### **Problem 4: Handle Anomalous Data**


In [43]:
# Xu ly cac gia tri khong hop le trong Dataframe thu 1
df=df.rename(columns={
    "Unnamed: 0": "STT",
    "sbd": "SBD",
    "toan": "Toán",
    "ngu_van": "Ngữ Văn",
    "vat_li": "Vật Lý",
    "hoa_hoc": "Hóa Học",
    "sinh_hoc": "Sinh Học",
    "lich_su": "Lịch Sử",
    "dia_li": "Địa Lý",
    "gdcd": "GDCD",
    "ngoai_ngu": "Ngoại Ngữ"
})
mon_thi = ["toan", "ngu_van", "vat_li", "hoa_hoc", "sinh_hoc", "lich_su", "dia_li", "gdcd", "ngoai_ngu"]
# xoa cot ten ngay sinh gioi tinh
df = df.drop(columns=["ten", "ngay_sinh", "gioi_tinh"], errors='ignore')
for mon in mon_thi:
    if mon in df.columns:
        df[mon] = df[mon].apply(lambda x: x if 0 <= x <= 10 else 0)
df.drop_duplicates(inplace=True)
df

,STT,SBD,Toán,Ngữ Văn,Vật Lý,Hóa Học,Sinh Học,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ
0,0,18014547,6.4,6.75,0.0,0.0,0.0,4.75,7.00,6.50,4.2
1,1,18014530,7.6,6.00,0.0,0.0,0.0,3.75,7.75,7.75,2.8
2,2,18014521,4.8,4.75,0.0,0.0,0.0,4.00,6.50,0.00,0.0
3,3,18014517,8.0,7.00,0.0,0.0,0.0,8.25,8.00,9.50,5.8
4,4,18014523,8.2,6.50,8.0,8.5,5.0,0.00,0.00,0.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...
870481,870481,17014593,4.6,4.00,0.0,0.0,0.0,3.75,6.75,6.25,2.0
870482,870482,17014597,7.4,8.00,0.0,0.0,0.0,5.50,7.50,9.50,2.6
870483,870483,17014595,8.2,8.75,0.0,0.0,0.0,4.50,6.50,8.50,6.2
870484,870484,17014598,4.6,6.50,0.0,0.0,0.0,4.50,7.50,9.00,3.6


In [44]:
df1 = df1.copy()  
df1=df1.rename(columns={
    "Unnamed: 0": "STT",
    "sbd": "SBD",
    "toan": "Toán",
    "ngu_van": "Ngữ Văn",
    "vat_li": "Vật Lý",
    "hoa_hoc": "Hóa Học",
    "sinh_hoc": "Sinh Học",
    "lich_su": "Lịch Sử",
    "dia_li": "Địa Lý",
    "gdcd": "GDCD",
    "ngoai_ngu": "Ngoại Ngữ"
})
for mon in mon_thi:
    if mon in df1.columns:
        df1[mon] = df1[mon].apply(lambda x: x if 0 <= x <= 10 else 0)
df1.drop_duplicates(inplace=True)
df1

,SBD,Toán,Ngữ Văn,Ngoại Ngữ,Vật Lý,Hóa Học,Sinh Học,Lịch Sử,Địa Lý,GDCD
0,1000001,3.6,5.00,4.0,0.0,0.0,0.0,2.75,6.00,8.75
1,1000002,8.4,6.75,7.6,0.0,0.0,0.0,8.50,7.50,8.25
2,1000003,5.8,7.50,5.0,0.0,0.0,0.0,7.25,5.50,8.75
3,1000004,7.4,7.50,8.6,0.0,0.0,0.0,7.50,6.50,7.50
4,1000005,7.2,8.50,9.0,0.0,0.0,0.0,8.00,8.50,8.25
...,...,...,...,...,...,...,...,...,...,...
995436,64006584,8.4,6.75,4.6,0.0,0.0,0.0,6.50,6.75,9.00
995437,64006585,5.6,6.50,2.8,0.0,0.0,0.0,6.25,6.75,8.50
995438,64006586,5.8,6.00,6.6,0.0,0.0,0.0,7.25,8.00,8.00
995439,64006587,7.6,6.75,7.0,0.0,0.0,0.0,8.75,7.25,9.75


In [45]:
df2=df2.rename(columns={
    "sbd": "SBD",
    "toan": "Toán",
    "ngu_van": "Ngữ Văn",
    "vat_li": "Vật Lý",
    "hoa_hoc": "Hóa Học",
    "sinh_hoc": "Sinh Học",
    "lich_su": "Lịch Sử",
    "dia_li": "Địa Lý",
    "gdcd": "GDCD",
    "ngoai_ngu": "Ngoại Ngữ",
    "ma_ngoai_ngu": "Mã Ngoại Ngữ"
})
df2 = df2.copy()  
for mon in mon_thi:
    if mon in df2.columns:
        df2[mon] = df2[mon].apply(lambda x: x if 0 <= x <= 10 else 0)
df2.drop_duplicates(inplace=True)
df2

,SBD,Toán,Ngữ Văn,Ngoại Ngữ,Vật Lý,Hóa Học,Sinh Học,Lịch Sử,Địa Lý,GDCD,Mã Ngoại Ngữ
0,1000001,8.4,6.75,8.0,6.00,5.25,5.0,0.00,0.00,0.00,N1
1,1000002,8.6,8.50,7.2,0.00,0.00,0.0,7.25,6.00,8.00,N1
2,1000003,8.2,8.75,8.2,0.00,0.00,0.0,7.25,7.25,8.75,N1
3,1000004,4.8,8.50,7.4,0.00,0.00,0.0,7.00,6.00,7.50,N1
4,1000005,8.6,9.00,7.8,0.00,0.00,0.0,9.00,8.75,8.50,N1
...,...,...,...,...,...,...,...,...,...,...,...
1061600,64007282,4.2,6.00,3.0,0.00,0.00,0.0,5.25,6.25,7.50,N1
1061601,64007283,5.2,6.00,3.6,0.00,0.00,0.0,6.25,5.75,7.75,N1
1061602,64007284,7.2,8.00,7.0,6.25,7.00,8.0,0.00,0.00,0.00,N1
1061603,64007285,5.2,5.25,4.4,0.00,0.00,0.0,4.75,6.50,8.75,N1


### **Merge Dataset**


In [46]:
import pandas as pd
# Gộp df và df1 theo cột "SBD"
merged_df = pd.merge(df, df1, on="SBD", how="outer", suffixes=('_2020', '_2022'))
# Gộp tiếp với df2 theo cột "SBD"
merged_df = pd.merge(merged_df, df2, on="SBD", how="outer", suffixes=('', '_2024'))
# Kiểm tra dữ liệu sau khi gộp
print(merged_df.head())


        STT      SBD  Toán_2020  Ngữ Văn_2020  Vật Lý_2020  Hóa Học_2020  \
0  173247.0  1000001        7.0           6.0         0.00          0.00   
1  173272.0  1000002        9.0           6.0         7.25          8.75   
2  173251.0  1000003        8.8           6.5         8.25          8.75   
3  173252.0  1000004        7.0           0.0         0.00          6.00   
4  173285.0  1000005        9.0           7.5         0.00          0.00   

   Sinh Học_2020  Lịch Sử_2020  Địa Lý_2020  GDCD_2020  ...  Toán  Ngữ Văn  \
0           0.00          8.00         0.00       0.00  ...   8.4     6.75   
1           7.25          0.00         0.00       0.00  ...   8.6     8.50   
2           4.75          0.00         0.00       0.00  ...   8.2     8.75   
3           0.00          0.00         0.00       0.00  ...   4.8     8.50   
4           0.00          5.75         7.25       9.25  ...   8.6     9.00   

   Ngoại Ngữ  Vật Lý  Hóa Học  Sinh Học  Lịch Sử  Địa Lý  GDCD  Mã Ngoại N

### **Store Dataset**


In [47]:
df.to_csv("diem_thi_2020.csv", index=False, encoding="utf-8-sig")
df1.to_csv("diem_thi_2022.csv", index=False, encoding="utf-8-sig")
df2.to_csv("diem_thi_2024.csv", index=False, encoding="utf-8-sig")

print(" Đã lưu các file CSV thành công!")

 Đã lưu các file CSV thành công!
